In [70]:
import pandas as pd
import os
import pickle
import torch
import numpy as np
import IPython.display as ipd

sub_dev = pd.read_csv('/Users/vigi99/AirtelDEV/MixMatch-pytorch/submission/submission_split_train_dev.csv', index_col='fname')
t_dev = pd.read_csv('/Users/vigi99/AirtelDEV/MixMatch-pytorch/data/freesound/train_curated.csv_dev', index_col='fname')
t_test = pd.read_csv('/Users/vigi99/AirtelDEV/MixMatch-pytorch/data/freesound/train_curated.csv_test', index_col='fname')
t_train = pd.read_csv('/Users/vigi99/AirtelDEV/MixMatch-pytorch/data/freesound/train_curated.csv_train', index_col='fname')
target_dev = pd.concat([t_dev, t_test])
lb = pickle.load(open('/Users/vigi99/AirtelDEV/MixMatch-pytorch/result/lb.pk' , 'rb'))
test_path = os.path.abspath('/Users/vigi99/kaggle/freesound/data/train_curated')
join_df = sub_dev.join(target_dev)

In [33]:
target_labels = lb.transform([label.split(',') for label in join_df['labels'].values])
predicted_labels = join_df[lb.classes_].values

In [34]:
import sys
sys.path.append(os.path.abspath('../utils'))
from eval import lwlrap_accumulator

lacc = lwlrap_accumulator()
lacc.accumulate_samples(torch.tensor(target_labels), torch.tensor(predicted_labels))

In [35]:
labels = np.array(lb.classes_)
scores = np.array(lacc.per_class_lwlrap())
num_examples = np.array(lacc._per_class_cumulative_count)
idx = np.argsort(scores)
sorted_labels = labels[idx].tolist()
sorted_scores = scores[idx].tolist()
sorted_numexamples = num_examples[idx].tolist()
for label, score, num_example in zip(sorted_labels, sorted_scores, sorted_numexamples):
    print(f"{label} = {score} with num of samples {num_example}")

Hiss = 0.2692307692307692 with num of samples 2
Computer_keyboard = 0.3451388888888889 with num of samples 4
Keys_jangling = 0.44791666666666663 with num of samples 4
Zipper_(clothing) = 0.47023809523809523 with num of samples 7
Tap = 0.5606060606060606 with num of samples 6
Walk_and_footsteps = 0.593095238095238 with num of samples 10
Gasp = 0.632034632034632 with num of samples 7
Squeak = 0.6367424242424242 with num of samples 8
Fill_(with_liquid) = 0.6854166666666666 with num of samples 8
Female_singing = 0.7061224489795919 with num of samples 7
Fart = 0.7135416666666666 with num of samples 6
Bathtub_(filling_or_washing) = 0.7181372549019608 with num of samples 6
Meow = 0.7365384615384616 with num of samples 11
Buzz = 0.7461028192371476 with num of samples 9
Cutlery_and_silverware = 0.7471264367816092 with num of samples 5
Yell = 0.75 with num of samples 6
Waves_and_surf = 0.75 with num of samples 9
Writing = 0.7569444444444444 with num of samples 10
Slam = 0.7577160493827161 with n

In [110]:
def get_examples(df, classname):
    return df[df.labels.str.contains(classname)]

def labels_values(df, lb, classname):
    predicted_labels = df.sample(1)[lb.classes_]
    file_path = os.path.join(test_path, predicted_labels.index.values[0])
    print(f"file was {file_path}")
    top_5 = np.argsort(predicted_labels.values[0])[-5:].tolist()
    top_5.reverse()
    print(np.array(predicted_labels.columns)[top_5])
    print(np.array(predicted_labels.values)[0][top_5])
    print(f"Value for {classname} was {predicted_labels[classname].values[0]}")
    return file_path

classname = 'Hiss'
example_for_class = get_examples(join_df, classname)
file_path = labels_values(example_for_class, lb, classname)
ipd.Audio(file_path)

file was /Users/vigi99/kaggle/freesound/data/train_curated/3d8da97d.wav
['Waves_and_surf' 'Traffic_noise_and_roadway_noise' 'Motorcycle' 'Stream'
 'Bus']
[0.38639459 0.15425353 0.14797546 0.14310554 0.12263963]
Value for Hiss was 0.027525754645466805


In [113]:
example_hiss = get_examples(t_train, 'Hiss').sample(1)
example_train_path = os.path.join(test_path, example_hiss.index.values[0])
print(example_hiss)
ipd.Audio(example_train_path)

             labels
fname              
68dbf3f8.wav   Hiss


In [85]:
from collections import Counter
samples = {}
for labels in t_train.labels.values:
    for label in labels.split(","):
        samples[label] = samples.get(label, 0) + 1
counter = Counter(samples)

In [103]:
counter.most_common()[-2:]

[('Fill_(with_liquid)', 42), ('Gasp', 41)]